Hello everyone,

In this project, it is required to implement an Arabic Font Identification System that will be able to identify the font for a given Arabic text snippet.
We will work with the [(ACdb) Arabic Calligraphy Database](https://drive.google.com/file/d/1dC7pwzT_RHL9B42H8-Nzf5Sant-86NV6/view) containing 9 categories of computer printed Arabic text snippets.

At first, what are the main steps we should go through in this project?

# TODOs:

1. Understand the problem (look at the big picture)
2. Get the data
3. Explore the data
4. Preprocessing
5. Extract features  
6. Style Recognition
7. Result

# 1. Understand the problem

*What is the objective?* Despite this question appears so easy, but it is very important..

Here, the objective is --

So, we need to build a predictive model with:

- Input: --
- Output: --

# 2. Get the data

After downloading and uncompressing the dataset, we will import it using --:

# 3. Explore the data


# 4. Preprocessing


# 5. Extract features


# 6. Styles Recognition


# 7. Result